In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Load the dataset
data = pd.read_csv("C:\\Users\\prath\\Downloads\\Fake.csv")  # Update the path to the dataset file

# Combine the 'title' and 'text' columns into one text column
data['text'] = data['title'] + " " + data['text']

# Remove unnecessary columns
data = data[['text', 'subject']]

# Data cleaning
def clean_text(text):
    text = re.sub(r'\S*@\S*\s?', '', text)  # Remove emails
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert to lowercase
    return text

data['text'] = data['text'].apply(clean_text)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['subject'], test_size=0.2, random_state=42)


In [7]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [8]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

y_pred = nb_classifier.predict(X_test_tfidf)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)

Accuracy:  0.5786672344049393
Confusion Matrix:
 [[  20    0   66    1   26  203]
 [   1   20   21   57   14   46]
 [   3    0 1741    0   34   43]
 [   5   70   26    8    5   46]
 [  15    0  199    0  123  560]
 [  22    0  356    0  160  806]]
Classification Report:
                  precision    recall  f1-score   support

Government News       0.30      0.06      0.10       316
    Middle-east       0.22      0.13      0.16       159
           News       0.72      0.96      0.82      1821
        US_News       0.12      0.05      0.07       160
      left-news       0.34      0.14      0.20       897
       politics       0.47      0.60      0.53      1344

       accuracy                           0.58      4697
      macro avg       0.36      0.32      0.31      4697
   weighted avg       0.51      0.58      0.52      4697

